# Import Module

In [16]:
import pandas as pd

# Import Data

In [17]:
name_list = ['State', 'Population', 'Seats', 'NA', 'Bid1', 'Bid2', 'Bid3', 'Bid4', 'Bid5', 'Bid6'] # list for the titles of dataframe retrieved
data_frame = pd.read_excel('Votes2.xlsx', 
                         header=3, 
                         names=name_list) # retrieve data from the fourth line 
data_frame.drop(columns='NA', axis=1, inplace=True) # drop the fourth column
data_array = data_frame.to_numpy()
voterank_data_frame = pd.read_excel('Votes2.xlsx', header=0, nrows=3,
                                    names=[x for x in range(10)])
voterank_data_frame.drop([x for x in range(4)], axis=1, inplace=True)
voterank_array = voterank_data_frame.to_numpy()
voterank_array_second = [1, 2, 0, 2, 0, 1] # the list of the second line, 0 represents Trump, 1 represents Clinton, 2 represents Bush

In [18]:
state_info_dict = dict()
for state_info_array in data_array:
  state_info_dict[state_info_array[0]] = [state_info_array[1:9]]

# Auxiliary Methods

In [19]:
def get_highest_vote_index(vote_list):
  """
  Get the highest vote index based on the list provided.
  """
  first, second, third = vote_list[0], vote_list[1], vote_list[2]

  # consider draw situation
  if first == second:
    if first < third:
      return 2
    elif first == third:
      return 'draw' + ' first second third'
    else:
      return 'draw' + ' first second'
  elif first == third:
    if first < second:
      return 1
    else:
      return 'draw' + ' first third'
  elif second == third:
    if first < second:
      return 'draw' + ' second third'
    else:
      return 0

  if first > second and first > third:
    return 0
  elif second > first and second > third:
    return 1
  else:
    return 2

def get_lowest_vote_index(vote_list):
  """
  Get the lowest vote index based on the list provided.
  """
  first, second, third = vote_list[0], vote_list[1], vote_list[2]

  # consider draw situation
  if first == second:
    if first < third:
      return 'draw' + ' first second'
    elif first == third:
      return 'draw' + ' first second third'
    else:
      return 2
  elif first == third:
    if first < second:
      return 'draw' + ' first third'
    else:
      return 1
  elif second == third:
    if first < second:
      return 0
    else:
      return 'draw' + ' second third'

  if first < second and first < third:
    return 0
  elif second < first and second < third:
    return 1
  else:
    return 2

def get_winner_message(winner_index):
  """
  Get winner massage based on the index provided, 0 represents Trump, 
  1 represents Clinton, 2 represents Bush.
  """
  if winner_index == 0:
    winner_message = 'Trump wins!'
  elif winner_index == 1:
    winner_message = 'Clinton wins!'
  else:
    winner_message = 'Bush wins!'
    
  return winner_message

# Current System

In [20]:
def current_system(state_info_dict):
  """
  Get the result based on current system.
  """
  vote_all_list = [0, 0, 0] # vote_Trump_total, vote_Clinton_total, vote_Bush_total
  # state vote
  for state_name, state_info in state_info_dict.items():
    state_info = state_info[0]
    vote_Trump = state_info[2] + state_info[3]
    vote_Clinton = state_info[4] + state_info[5]
    vote_Bush = state_info[6] + state_info[7]
    vote_list = [vote_Trump, vote_Clinton, vote_Bush] # (vote_Trump, vote_Clinton, vote_Bush)
    vote_all_list[get_highest_vote_index(vote_list)] += state_info[1]

  winner_index = get_highest_vote_index(vote_all_list)
  
  return get_winner_message(winner_index), vote_all_list
  
current_system(state_info_dict)

('Trump wins!', [228, 156, 153])

# Plurality rule for overall U.S.

In [21]:
def plurality_overall_us(state_info_dict):
  """
  Get the result based on the plurality rule for overall U.S.
  """
  vote_all_list = [0, 0, 0] # vote_Trump_total, vote_Clinton_total, vote_Bush_total
  # state vote
  for state_name, state_info in state_info_dict.items():
    state_info = state_info[0]
    vote_all_list[0] += state_info[2] + state_info[3]
    vote_all_list[1] += state_info[4] + state_info[5]
    vote_all_list[2] += state_info[6] + state_info[7]
  
  winner_index = get_highest_vote_index(vote_all_list)
  
  return get_winner_message(winner_index), vote_all_list

plurality_overall_us(state_info_dict)

('Clinton wins!', [99539464, 116739443, 108614095])

# Plurality with run-off in the whole U.S.

In [22]:
def plurality_with_runoff_whole_us(state_info_dict):
  """
  Get the result based on the plurality with run-off in the whole U.S.
  """
  vote_all_list = plurality_overall_us(state_info_dict)[1]
  lowest_index = get_lowest_vote_index(vote_all_list)
  vote_all_list[lowest_index] = 0
  
  for state_name, state_info in state_info_dict.items():
    state_info = state_info[0]
    index_first = voterank_array_second[2 * lowest_index]
    index_second = index_first + 1
    vote_all_list[index_first] += state_info[2 * lowest_index+2]
    vote_all_list[index_second] += state_info[2 * lowest_index+3]
    
  winner_index = get_highest_vote_index(vote_all_list)


  return get_winner_message(winner_index), vote_all_list

plurality_with_runoff_whole_us(state_info_dict)

('Bush wins!', [0, 134311912, 190581090])

# Plurality with run-off per state

In [23]:
def plurality_with_runoff_per_state(state_info_dict):
  """
  Get the result based on the plurality with run-off per state.
  """
  vote_all_list = plurality_overall_us(state_info_dict)[1]
  lowest_index = get_lowest_vote_index(vote_all_list)
  vote_all_list = [0, 0, 0]

  for state_name, state_info in state_info_dict.items():
    vote_list = [0, 0, 0]
    state_info = state_info[0]
    index_first = voterank_array_second[2 * lowest_index]
    index_second = index_first + 1
    vote_list[index_first] += state_info[2 * index_first+2] + state_info[2 * index_first+3]
    vote_list[index_second] += state_info[2 * index_second+2] + state_info[2 * index_second+3]
    vote_list[index_first] += state_info[2 * lowest_index+2]
    vote_list[index_second] += state_info[2 * lowest_index+3]

    vote_all_list[get_highest_vote_index(vote_list)] += state_info[1]

  winner_index = get_highest_vote_index(vote_all_list)

  return get_winner_message(winner_index), vote_all_list

plurality_with_runoff_per_state(state_info_dict)

('Bush wins!', [0, 0, 537])